## Using Data From Virtuoso in Tensorflow (SQL Query)

In [1]:
import pandas as pd
import pyodbc
import tensorflow as tf
import datetime

SHUFFLE_BUFFER = 500
BATCH_SIZE = 2

### Set Connection, and Create Dataframe

In [13]:
cnxn = pyodbc.connect("DSN=Local Virtuoso;UID=dba;pwd=dba")

q = 'SELECT * FROM "tensorflow"."diabetes".data'

d = pd.read_sql_query(q,cnxn)


In [14]:
df = pd.DataFrame(d)

outcome = df.pop("Outcome")

df.head()



,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [15]:
feat_names = df.columns.tolist()
feats = df[feat_names]


### Convert to Tensor and continue in TensorFlow

In [16]:
print(tf.convert_to_tensor(feats))

normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(feats)
normalizer(feats.iloc[:3])

2021-11-30 11:12:35.345980: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


tf.Tensor(
[[  6.    148.     72.    ...  33.6     0.627  50.   ]
 [  1.     85.     66.    ...  26.6     0.351  31.   ]
 [  8.    183.     64.    ...  23.3     0.672  32.   ]
 ...
 [  2.    122.     70.    ...  36.8     0.34   27.   ]
 [  5.    121.     72.    ...  26.2     0.245  30.   ]
 [  1.    126.     60.    ...  30.1     0.349  47.   ]], shape=(767, 8), dtype=float64)


2021-11-30 11:12:35.708140: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


<tf.Tensor: shape=(3, 8), dtype=float32, numpy=
array([[ 0.6387271 ,  0.84705454,  0.14960383,  0.90778947, -0.6935593 ,
         0.20362139,  0.4676379 ,  1.4246367 ],
       [-0.8458293 , -1.1243613 , -0.16038112,  0.5315605 , -0.6935593 ,
        -0.6842581 , -0.36549374, -0.1917781 ],
       [ 1.2325497 ,  1.9422855 , -0.26370946, -1.2868794 , -0.6935593 ,
        -1.1028302 ,  0.6034746 , -0.10670363]], dtype=float32)>

In [44]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(30, activation='relu'),
    tf.keras.layers.Dense(30, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

model = get_basic_model()

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(feats, outcome, epochs=50, batch_size=BATCH_SIZE, callbacks=[tensorboard_callback])

2021-11-30 12:04:44.181802: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-11-30 12:04:44.181816: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-11-30 12:04:44.181887: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


Epoch 1/50
253/384 [==================>...........] - ETA: 0s - loss: 0.5963 - accuracy: 0.6739

2021-11-30 12:04:44.534617: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-11-30 12:04:44.534629: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-11-30 12:04:44.553482: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-11-30 12:04:44.554256: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-11-30 12:04:44.555131: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/fit/20211130-120444/train/plugins/profile/2021_11_30_12_04_44

2021-11-30 12:04:44.555665: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/fit/20211130-120444/train/plugins/profile/2021_11_30_12_04_44/Daniels-MBP-2.fios-router.home.trace.json.gz
2021-11-30 12:04:44.557046: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/fit/20211130-1

384/384 [==============================] - 1s 758us/step - loss: 0.5728 - accuracy: 0.7001
Epoch 2/50
384/384 [==============================] - 0s 661us/step - loss: 0.4932 - accuracy: 0.7497
Epoch 3/50
384/384 [==============================] - 0s 679us/step - loss: 0.4711 - accuracy: 0.7627
Epoch 4/50
384/384 [==============================] - 0s 673us/step - loss: 0.4576 - accuracy: 0.7771
Epoch 5/50
384/384 [==============================] - 0s 669us/step - loss: 0.4509 - accuracy: 0.7784
Epoch 6/50
384/384 [==============================] - 0s 667us/step - loss: 0.4426 - accuracy: 0.7836
Epoch 7/50
384/384 [==============================] - 0s 678us/step - loss: 0.4376 - accuracy: 0.7797
Epoch 8/50
384/384 [==============================] - 0s 664us/step - loss: 0.4322 - accuracy: 0.7940
Epoch 9/50
384/384 [==============================] - 0s 674us/step - loss: 0.4286 - accuracy: 0.7927
Epoch 10/50
384/384 [==============================] - 0s 668us/step - loss: 0.4231 - accurac

In [45]:
!mkdir -p saved_model
model.save('saved_model/my_model')

2021-11-30 12:11:46.101320: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: saved_model/my_model/assets
